In [1]:
! pip install s3fs
! PYTHONWARNINGS="ignore"

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
'PYTHONWARNINGS' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from collections import defaultdict
from datetime import datetime
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import requests
import us
import shutil
from typing import Dict, List, Optional
import warnings
%matplotlib inline

warnings.filterwarnings("ignore")
pd.options.display.max_rows = 999

logging.basicConfig()
log = logging.getLogger()
log.setLevel(logging.INFO)

In [3]:
DELPHI_BUCKET_NAME = 'covid19-lake' 
COVIDCAST_PREFIX = "covidcast/json"

In [4]:
def _get_unsigned_s3_client():
    return boto3.client('s3', config=Config(signature_version=UNSIGNED))

def get_latest_delphi_files(bucket_name: str=DELPHI_BUCKET_NAME, 
                            prefix: Optional[str]=COVIDCAST_PREFIX) -> List[str]:
    """
    Given an s3 bucket name and optional path prefix, fetch all file names matching that prefix.

    """
    s3 = _get_unsigned_s3_client()
    paginator = s3.get_paginator('list_objects')
    # The bucket has a ton of stuff and depending on the prefix value you 
    # choose you may exceed the max list_objects return size (1000). 
    #The paginator allows you always fetch all of the file paths. 
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    s3_keys = []
    for page in page_iterator:
        s3_keys.extend(x["Key"] for x in page["Contents"])
    return s3_keys

def group_covidcast_files_by_source(s3_keys: List[str]) -> Dict[str, List[str]]:
    """
    The delphi public s3 bucket contains files with paths that look like this, 
    'covidcast/json/data/consensus/part-00000-64b3ef4a-f21d-4ff8-8993-80e9447b3e42-c000.json'
    
    Given an array of keys, this function groups all the file parts for a given data type/data source
    for further processing.
    
    There is an additional metadatafile which is not includeded. 
    
    """
    files_by_type = defaultdict(list)
    for path in s3_keys:
        data_type = path.split("/")[3]
        if data_type != "metadata.json":
            files_by_type[data_type].append(path)
    return files_by_type

def get_delphi_covidcast_metadata(bucket_name: str=DELPHI_BUCKET_NAME) -> pd.DataFrame:
    covidcast_metadata_path = "s3://" + DELPHI_BUCKET_NAME + "/" +  'covidcast/json/metadata/metadata.json'
    metadata_df = pd.read_json(covidcast_metadata_path, lines=True)
    
    metadata_df.min_time = pd.to_datetime(metadata_df.min_time, format="%Y%m%d")
    metadata_df.max_time = pd.to_datetime(metadata_df.max_time, format="%Y%m%d")
    metadata_df.last_update = pd.to_datetime(metadata_df.last_update, unit='s')
    return metadata_df

In [5]:
keys = get_latest_delphi_files()
files_by_source = group_covidcast_files_by_source(keys)
covidcast_metadata = get_delphi_covidcast_metadata()

In [6]:
covidcast_metadata = get_delphi_covidcast_metadata()

# What data is there?
## look at the dedicated metadata file

In [7]:
covidcast_metadata.head(20)

,data_source,geo_type,last_update,max_time,max_value,mean_value,min_time,min_value,num_locations,signal,stdev_value,time_type
0,doctor-visits,county,2020-05-28 23:31:01,2020-05-25,87.447156,1.085951,2020-02-01,0.000000,2191,smoothed_adj_cli,2.127873,day
1,doctor-visits,hrr,2020-05-28 23:31:00,2020-05-25,25.875139,1.364887,2020-02-01,0.000000,306,smoothed_adj_cli,2.235863,day
2,doctor-visits,msa,2020-05-28 23:31:00,2020-05-25,20.865074,1.122883,2020-02-01,0.000000,378,smoothed_adj_cli,1.820549,day
3,doctor-visits,state,2020-05-28 23:31:00,2020-05-25,17.282437,1.457170,2020-02-01,0.000000,51,smoothed_adj_cli,1.960175,day
4,doctor-visits,county,2020-05-28 23:31:01,2020-05-25,71.334744,0.918419,2020-02-01,0.000000,2191,smoothed_cli,1.825289,day
5,doctor-visits,hrr,2020-05-28 23:31:00,2020-05-25,28.686304,1.227353,2020-02-01,0.000000,306,smoothed_cli,2.055017,day
6,doctor-visits,msa,2020-05-28 23:31:00,2020-05-25,22.024339,0.981958,2020-02-01,0.000000,378,smoothed_cli,1.613278,day
7,doctor-visits,state,2020-05-28 23:31:00,2020-05-25,20.202046,1.479935,2020-02-01,0.000000,51,smoothed_cli,2.038033,day
8,fb-survey,county,2020-06-01 15:12:30,2020-05-31,5.080565,0.516699,2020-04-06,0.000000,641,raw_cli,0.501254,day
9,fb-survey,hrr,2020-06-01 15:12:34,2020-05-31,4.550654,0.515073,2020-04-06,0.000000,306,raw_cli,0.467320,day


In [8]:
covidcast_metadata[['data_source', 'signal', 'time_type','geo_type']].drop_duplicates()

,data_source,signal,time_type,geo_type
0,doctor-visits,smoothed_adj_cli,day,county
1,doctor-visits,smoothed_adj_cli,day,hrr
2,doctor-visits,smoothed_adj_cli,day,msa
3,doctor-visits,smoothed_adj_cli,day,state
4,doctor-visits,smoothed_cli,day,county
5,doctor-visits,smoothed_cli,day,hrr
6,doctor-visits,smoothed_cli,day,msa
7,doctor-visits,smoothed_cli,day,state
8,fb-survey,raw_cli,day,county
9,fb-survey,raw_cli,day,hrr


Observations
* multiple data types available
* multiple signals per datatype
* each data type appears to have both county and state-level data. 

In [9]:
def cache_data_locally(s3_keys: List[str], local_folder: str, bucket_name=DELPHI_BUCKET_NAME) -> None:
    """
    Downloads json files from s3 
    """
    local_folder = pathlib.Path(local_folder)

    shutil.rmtree(local_folder, ignore_errors=True)
    local_folder.mkdir(parents=True)
        
    s3 = _get_unsigned_s3_client()
    for key in s3_keys:
        log.info(f"Downloading file s3://{bucket_name}/{key}")
        filename=pathlib.Path(key).name
        local_file = local_folder/filename
        s3.download_file(bucket_name, key, str(local_file))


# Cache all data locally

In [ ]:
for data_source, keys in files_by_source.items():
    if not data_source.startswith("jhu"):
        log.info(f"Caching {len(keys)} {data_source} files locally.")
        cache_data_locally(keys, f"covidcast/{data_source}")

INFO:root:Caching 3 consensus files locally.
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/consensus/part-00000-fa5146fd-8ebe-401b-b713-438ff4192272-c000.json
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/consensus/part-00001-fa5146fd-8ebe-401b-b713-438ff4192272-c000.json
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/consensus/part-00002-fa5146fd-8ebe-401b-b713-438ff4192272-c000.json
INFO:root:Caching 9 doctor-visits files locally.
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/doctor-visits/part-00000-47eb354a-2970-4dea-89a5-2d3ce65b0789-c000.json
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/doctor-visits/part-00001-47eb354a-2970-4dea-89a5-2d3ce65b0789-c000.json
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/doctor-visits/part-00002-47eb354a-2970-4dea-89a5-2d3ce65b0789-c000.json
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/doctor-visits/part-00003-47eb35

INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/fb-survey/part-00049-b592480c-62f5-4d81-8a9b-638b6b1ad448-c000.json
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/fb-survey/part-00050-b592480c-62f5-4d81-8a9b-638b6b1ad448-c000.json
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/fb-survey/part-00051-b592480c-62f5-4d81-8a9b-638b6b1ad448-c000.json
INFO:root:Downloading file s3://covid19-lake/covidcast/json/data/fb-survey/part-00052-b592480c-62f5-4d81-8a9b-638b6b1ad448-c000.json


In [ ]:
! ls covidcast

In [ ]:
# Load data from source into dataframes

def construct_combined_dataframe(local_folder: str) -> pd.DataFrame:
    combined_df = pd.DataFrame()
    for f in glob.glob(f"{local_folder}/*.json"):
        part_df = pd.read_json(f, lines=True)
        part_df.time_value = pd.to_datetime(part_df.time_value, format="%Y%m%d")
        combined_df = combined_df.append(part_df, ignore_index=True)
    return combined_df

# Quick plots of all signals 

In [ ]:
common_target_columns = ["time_value", "geo_value", "geo_type", "signal", "value"]
target_data_sources = [x.name for x in pathlib.Path("./covidcast/").iterdir() if x.is_dir()]
target_data_sources

In [ ]:
state='ny'
for source in target_data_sources:
    fig, ax = plt.subplots()
    source_df = construct_combined_dataframe(f"covidcast/{source}")
    source_df = source_df[source_df.geo_value==state]
    groups = source_df.groupby("signal")
    for name, group in groups:
        group.plot(x='time_value', y='value', label=f"{source}_{name}", ax=ax, kind='line')


# Consensus Data source

In [ ]:
consensus_df = construct_combined_dataframe("covidcast/consensus")

In [ ]:
consensus_df.signal.unique()

In [ ]:
consensus_df[consensus_df.geo_type=='state']

In [ ]:
consensus_df[consensus_df.geo_value=="ny"].plot(x="time_value", y="value")

In [ ]:
consensus_df[consensus_df.geo_value=="ca"].plot(x="time_value", y="value")

# Doctor Visits

In [ ]:

doctor_visits_df = construct_combined_dataframe("covidcast/doctor-visits")

In [ ]:
doctor_visits_df.signal.unique()

In [ ]:
state="ca"
groups = doctor_visits_df[doctor_visits_df.geo_value==state].groupby("signal")
fig, ax = plt.subplots()
for name, group in groups: 
    group.plot(x="time_value", y="value", title=f"{state} doctor visits", label=name, ax=ax)

# Indicator Combination

In [ ]:
indicator_combination_df = construct_combined_dataframe("covidcast/indicator-combination")